In [1]:
import numpy as np
import json
import tensorflow as tf
import os
import sys

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=256)])
  except RuntimeError as e:
    print(e)
    
     
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
#from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy

from scan_gym import envs
seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

#import imp
#import test as nntools
#imp.reload(nntools)
import csv
import json
import copy

from utils import policy_test

GPUs:  1


In [2]:
os.getcwd()

'/home/pico/uni/romi/scanner_cube_2d'

In [3]:
models_path  = '/home/pico/uni/romi/scanner-gym_models'
models = ['207_2d','208_2d','209_2d','210_2d','211_2d']
n_images = 8
continuous = False

policy_name ='20220505-214345'
policies_path = '/home/pico/uni/romi/scanner_cube_2d/generated_data/policies'
policy = tf.compat.v2.saved_model.load(os.path.join(policies_path,policy_name))

In [4]:
scan_env = suite_gym.load('ScannerEnv-v1',gym_kwargs={'models_path':models_path, 'train_models':[models[0]],
                                                   'n_images':n_images, 'continuous':False,
                                                   'gt_mode':True,'cube_view':'static'})
        
tf_env = tf_py_environment.TFPyEnvironment( scan_env )

In [5]:
policy_test.test_policy(models_path=models_path, models=models, policy=policy,
                        n_images=n_images, n_episodes = 5, dest_path='./aqui.json' )

plant-207_2d 4     
207_2d rwd_mean 0.3647 rwd_std 0.0414 gt_ratio_mean 0.5750 gt_ratio_std 0.0562
plant-208_2d 4     
208_2d rwd_mean 0.1577 rwd_std 0.0650 gt_ratio_mean 0.2883 gt_ratio_std 0.0592
plant-209_2d 4     
209_2d rwd_mean 0.4502 rwd_std 0.0378 gt_ratio_mean 0.5304 gt_ratio_std 0.0479
plant-210_2d 4     
210_2d rwd_mean 0.1828 rwd_std 0.0545 gt_ratio_mean 0.3056 gt_ratio_std 0.0629
plant-211_2d 4     
211_2d rwd_mean 0.5028 rwd_std 0.1645 gt_ratio_mean 0.5650 gt_ratio_std 0.1475


In [6]:
actions, images, theta_bias, rewards, t_rwd, gt_ratio = policy_test.run_episode(scan_env,tf_env,policy)

In [6]:
print(actions, images, theta_bias,rewards, t_rwd, gt_ratio )

[16, 5, 19, 12, 7, 7, 18] [[14, 2], [39, 2], [84, 0], [89, 3], [91, 2], [96, 1], [101, 1], [103, 3]] 71 [0.01827236, 0.08378537, 0.1683731, 0.052878782, 0.01866232, 0.0116907125, 0.0050452724] 0.3587079126138226 0.5513073005093378


In [11]:
action_step = policy.action(state)
state = tf_env.step(action_step.action)

In [14]:
state.reward.numpy()[0]

0.01827236

In [ ]:
def run_episode(env,tf_env,policy):
    state = tf_env.reset()
    time_steps = 40
    actions = []
    images = []
    rewards = []
    images.append([env.current_theta,env.current_phi])
    for i in range(1,time_steps):
        action_step = policy.action(state)
        actions.append(int(action_step.action.numpy()[0]))
        state = tf_env.step(action_step.action)
        images.append([env.current_theta,env.current_phi])
        if state.is_last():
            break
    return actions, images, env.theta_bias, env.total_reward , env.last_gt_ratio

In [ ]:
data_template = {'actions':[],'images':[],'position_bias':[],'return':[],'gt_ratio':[],'empty_vox':[],
                 'ind_vox':[],'solid_vox':[],'gt_ratio_mean':0.0,'gt_ratio_std':0}

In [ ]:
def run_episode(env,tf_env,policy):
    state = tf_env.reset()
    time_steps = 40
    actions = []
    for i in range(1,time_steps):
        action_step = policy.action(state)
        actions.append(int(action_step.action.numpy()[0]))
        state = tf_env.step(action_step.action)
        if state.is_last():
            break
    return actions, env.kept_images, env.position_bias, env.total_reward , env.last_gt_ratio , env.h

In [ ]:
policy_name ='20220505-214345'
policies_path = '/home/pico/uni/romi/scanner_cube_2d/generated_data/policies'
models_path  = '/home/pico/uni/romi/scanner-gym_models'
models = ['207_2d','208_2d','209_2d','210_2d','211_2d']
n_images = 8

policy = tf.compat.v2.saved_model.load(os.path.join(policies_path,policy_name))

In [ ]:
collected_data = {}

for p in plant_models:
    #load environment with selected plant model
    env = suite_gym.load('ScannerEnv-v1',
    gym_kwargs={'dataset_path':os.path.join(plant_models_path,str(p).zfill(3)),'gt_mode':True,'init_pos_inc_rst':True}) 
    tf_env = tf_py_environment.TFPyEnvironment( env )
    #run 180 times (rotating model one step each time)
    data_holder = copy.deepcopy(data_template)
    
    for i in range(180):
        actions, images, position_bias, t_return, gt_ratio , histogram = run_episode(env,tf_env,policy)
        data_holder['actions'].append(actions)
        data_holder['images'].append(images)
        data_holder['position_bias'].append(position_bias)
        data_holder['return'].append(t_return)
        data_holder['gt_ratio'].append(gt_ratio)
        data_holder['empty_vox'].append(histogram[0])
        data_holder['ind_vox'].append(histogram[1])
        data_holder['solid_vox'].append(histogram[2])
        print("\rplant-{} {}     ".format(str(p).zfill(3), i), end="")
    
    data_holder['gt_ratio_mean'] = float(np.mean(data_holder['gt_ratio']))
    data_holder['gt_ratio_std'] = float(np.std(data_holder['gt_ratio']))
    collected_data[str(p).zfill(3)] = copy.deepcopy(data_holder)
    print("\rplant-{} gt_mean {}   gt_std {}            ".format(str(p).zfill(3),data_holder['gt_ratio_mean'],data_holder['gt_ratio_std']))

In [ ]:
dest_path = '/home/pico/uni/romi/desktop_scanner_cube_repo/scanner_cube'
f = open(os.path.join(dest_path,"test_" + policy_name+ ".json"),'w')
json.dump(collected_data, f)
f.close()

In [ ]:
with open('/home/pico/uni/romi/scanner_cube/uni_rnd_policy_runs_10__.json') as f:
    comp = json.load(f)

In [ ]:
for p in plant_models:
    plant = str(p).zfill(3)
    print("plant-{} uni_mean {:.5f} uni_std {:.5f} rnd_mean {:.5f} rnd_std {:.5f}".format(plant,
            comp[plant]['uni']['gt_mean'], comp[plant]['uni']['gt_std'],
            comp[plant]['rnd']['gt_mean'], comp[plant]['rnd']['gt_std']))

In [ ]:
def run_episode(env,tf_env,policy):
    state = tf_env.reset()
    time_steps = 40
    actions = []
    for i in range(1,time_steps):
        action_step = policy.action(state)
        actions.append(int(action_step.action.numpy()[0]))
        state = tf_env.step(action_step.action)
        if state.is_last():
            break
    return actions, env.kept_images, env.position_bias, env.total_reward , env.last_gt_ratio , env.hb

In [ ]:
scan_env = suite_gym.load('ScannerEnv-v1',gym_kwargs={'models_path':models_path, 'train_models':['206_2d'],
                                                   'n_images':n_images, 'continuous':False,
                                                   'gt_mode':True,'cube_view':'static'})

tf_env = tf_py_environment.TFPyEnvironment( scan_env )

In [ ]:
tf_env.reset(theta_init=0)